# **Breast Cancer Classification by Using Iterative Soft Thresholding for LASSO**

Course: Math 535 - Mathematical Methods in Data Science (MMiDS) - Spring 2024

Name: Junyi Liu; Weibing Wang

# **1 - Background and motivation**


Breast cancer remains one of the most common and devastating forms of cancer worldwide, affecting millions of women each year.

The advent of genomic technologies has revolutionized cancer research, enabling the analysis of vast arrays of genetic data to uncover biomarkers and genetic mutations associated with breast cancer. However, the high dimensionality of genomic data presents significant challenges. Typical datasets may contain expressions of tens of thousands of genes, yet only a small fraction of these genes are relevant for diagnosing or understanding the pathology of breast cancer. Diagnosing breast cancer using traditional least squares methods can easily lead to overfitting problems. This scenario necessitates robust feature selection methods to identify the most informative genetic features for binary classification.

Linear models with L1 regularization, commonly known as LASSO (Least Absolute Shrinkage and Selection Operator), have emerged as a powerful tool for feature selection in high-dimensional datasets. However, the computational complexity of applying LASSO in very high-dimensional spaces and the challenge of tuning the regularization parameter, which dictates the sparsity of the solution, can be prohibitive. Therefore we use ISTA (Iterative Shrinkage-Thresholding Algorithm) which applies the gradient descent and soft thresholding operator to solve the LASSO regression problem.




# **2 - Theory**




**2.1 LASSO** (Least Absolute Shrinkage and Selection Operator)


Before discussing the ISTA, let us first clarify the objective function of LASSO, as ISTA is primarily designed to optimize this objective function.

The objective function for LASSO regression is given by:
\begin{equation}
    \min_{\beta} \left( \frac{1}{2n} \|y - X\beta\|^2_2 + \lambda \|\beta\|_1 \right)
\end{equation}
where:
- $y$ is the response variable vector.
- $X$ is the design matrix containing features of observations.
- $\beta$ is the coefficient vector that needs to be solved.
- $\|y - X\beta\|_2^2$, represents the sum of squared residuals, indicating the model error, similar to the least squares.
- $\|\lambda \beta\|_1$ is the L1 regularization term controlled by the regularization parameter $\lambda$  , which is used to penalize the sum of the absolute values of the coefficients to control the complexity of the model, some coefficient values need to be reduced to zero. Such treatment aids in feature selection, automatically identifying the features that contribute most to the model while suppressing those that are irrelevant to avoid overfitting.
- $\lambda$ is the regularization parameter that controls the strength of the L1 regularization term.
- $n$ is the number of samples.


---
Consider a convex objective of the form

\begin{equation}
f(\theta) = L(\theta) + R(\theta)
\end{equation}

Here $L(\theta)$ is a convex and differentiable loss function, while $R(\theta)$ is a convex but possibly non-differentiable regularization term. When using ISTA, we typically use the unconstrained form of the LASSO problem where $L(\theta)$ is a squared loss term and $R(\theta) $is the L1 norm $\lambda \|\beta\|_1$. Due to the presence of the L1 term, directly applying standard gradient descent is not applicable because the L1 norm is not differentiable at zero. This introduces the concept of the proximal operator.


**2.2 Proximal Operator**

Proximal Operator is a specific mathematical tool used to solve optimization problems that contain non-differentiable function parts $R(\theta)$.

For a regularization term $R(\theta)$and a parameter $v$, the proximal operator is defined as:
\begin{equation}
\text{prox}_{R}(v) = \arg\min_{\theta} \left\{ R(\theta) + \frac{1}{2} \|\theta - v\|_2^2 \right\}
\end{equation}
Here, the goal is to find a point $\theta$ that minimizes the sum of the regularization term and the squared Euclidean distance between $\theta$ and $v$.



**2.3 Soft Thresholding Operator**

For L1 regularization, the proximal operator is specifically expressed through the soft thresholding operator, which is given by:

\begin{equation}
S_{\lambda}(\beta_i) = \text{sign}(\beta_i) \max(|\beta_i| - \lambda, 0)
\end{equation}

here $S_{\lambda}(\beta_i)$ represents the soft thresholding operation applied to each coefficient $\beta_i$, sign($\beta_i$) represents the Sign function:

\begin{equation}
\text{sign}(x) =
\begin{cases}
-1 & \text{if } x < 0 \\
0 & \text{if } x = 0 \\
1 & \text{if } x > 0
\end{cases}
\end{equation}


---

Derivation of Soft Thresholding Operator:
Considering the L1 regularization term $R(\theta) = \lambda\|\theta\|_1$, we want to compute its proximal operator. We face the following optimization problem:

\begin{equation}
\arg\min_{\theta} \left\{ \lambda\|\theta\|_1 + \frac{1}{2} \|\theta - v\|_2^2 \right\}
\end{equation}

Due to the additivity, for a single coefficient $\theta_i\$, the problem simplifies to:

\begin{equation}
\arg\min_{\theta_i} \left\{ \lambda|\theta_i| + \frac{1}{2} (\theta_i - v_i)^2 \right\}
\end{equation}

To minimize this function, we can consider the positive and negative values of $\theta_i$ separately. For $\theta_i > 0$, the the function becomes:

\begin{equation}
\arg\min_{\theta_i} \left\{ \lambda\theta_i + \frac{1}{2} (\theta_i - v_i)^2 \right\}
\end{equation}

We can minimize this function by setting its derivative to zero:

\begin{equation}
\lambda +  (\theta_i - v_i) = 0
\end{equation}
It is easy to notice that the second derivative is 1.

Solving gives:

\begin{equation}
\theta_i = v_i - \lambda
\end{equation}

Similarly, for the case $\theta_i < 0$, we get:

\begin{equation}
\theta_i = v_i + \lambda
\end{equation}

Combining these two cases, and considering that when $|v_i| \leq \lambda$. The absolute value function $|\theta_i|$ is not differentiable at 0, which means that the minimum point cannot be found directly by setting the derivative to 0, the optimal solution is $\theta_i = 0$, because setting $\theta_i$ to zero minimizes the function in this case, we arrive at the definition of the soft thresholding operator:

\begin{equation}
S_{\lambda}(\beta_i) = \text{sign}(\beta_i) \max(|\beta_i| - \lambda, 0)
\end{equation}


**2.4 Iterative Shrinkage-Thresholding Algorithm (ISTA)**

The objective function for LASSO regression is given by:
\begin{equation}
    \min_{\beta} \left( \frac{1}{2n} \|y - X\beta\|^2_2 + \lambda \|\beta\|_1 \right)
\end{equation}

The first term is the squared error term, and the second term is the L1 regularization term.

Each step of the ISTA can be divided into two main parts:

**Part 1: Gradient descent**

At each step, we perform gradient descent on the loss function $L(\beta)$ to update the coefficients. The update rule is given by:

\begin{equation}
\beta^{(k+1)} = \beta^{(k)} - \alpha \nabla L(\beta^{(k)})
\end{equation}
Here, $\alpha$ is the step size, and $\beta^{(k)}$ denotes the coefficient vector at the $k$-th iteration.

**Part 2: Soft Thresholding**

After updating the coefficients using gradient descent to obtain $\beta^{(k+1)}$, we apply the soft thresholding operation to each coefficient, yielding:

\begin{equation}
\beta_j^{(k+1)} = \text{sign}(\beta_j^{(k+1)}) \max(|\beta_j^{(k+1)}| - \lambda, 0)
\end{equation}

Here, $\lambda$ is the regularization parameter of the LASSO method, $\text{sign}(\beta_j)$ is the sign function.


---


The specific steps of the ISTA are as follows:

1. Initialize $\beta^{(0)}$ (usually with zeros).
2. For each iteration:


  *   Compute the gradient of the loss function $L(\beta^{(k)})$.
  *   Update the coefficients with gradient descent:
  \begin{equation}
  \beta^{(k+1)} = \beta^{(k)} - \alpha \nabla L(\beta^{(k)})
  \end{equation}

  *   Apply the soft-thresholding operator to each coefficient:
  \begin{equation}
  \beta_j^{(k+1)} \leftarrow \text{sign}(\beta_j^{(k+1)}) \max\left(|\beta_j^{(k+1)}| - \lambda, 0\right)
  \end{equation}

3. Convergence check: Terminate if the difference between successive iterations is below a threshold or if the maximum number of iterations has been reached.


# **3 - Implementation:**


In [13]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [14]:
df= pd.read_csv('Breast_classes2.csv')

print("datashape：", df.shape)

print("datatype：", df.dtypes)

datashape： (289, 35983)
datatype： samples                            object
type                               object
NM_144987                         float64
NM_013290                         float64
ENST00000322831                   float64
                                   ...   
lincRNA:chr9:4869500-4896050_F    float64
NM_016053                         float64
NM_001080425                      float64
ENST00000555638                   float64
ENST00000508993                   float64
Length: 35983, dtype: object


In [15]:
print(df.head())

                                         samples    type  NM_144987  \
0  GSM1823702_252800417016_S01_GE1_107_Sep09_1_2  normal   8.693318   
1  GSM1823703_252800417016_S01_GE1_107_Sep09_2_1  normal   9.375980   
2  GSM1823704_252800416877_S01_GE1_107_Sep09_2_3  normal   8.943442   
3  GSM1823705_252800416894_S01_GE1_107_Sep09_1_1  normal   9.020798   
4  GSM1823706_252800416894_S01_GE1_107_Sep09_1_3  normal   8.806154   

   NM_013290  ENST00000322831  NM_001625  lincRNA:chr7:226042-232442_R  \
0   7.718016         6.044438  10.747077                      9.133777   
1   7.072232         6.976741  10.429671                      9.526500   
2   7.964573         6.269055  10.825025                      9.396855   
3   7.824639         6.165165  11.646788                      8.776462   
4   7.555348         6.230969  11.635247                      8.911383   

   NM_032391  ENST00000238571  XR_108906  ...  \
0   4.735581         5.634732   4.670231  ...   
1   5.221089         5.425187 

In [16]:

type_mapping = {
    'normal': 0,
    'breast_adenocarcinoma': 1
}

df['type_encoded'] = df['type'].map(type_mapping)

print(df.head())

                                         samples    type  NM_144987  \
0  GSM1823702_252800417016_S01_GE1_107_Sep09_1_2  normal   8.693318   
1  GSM1823703_252800417016_S01_GE1_107_Sep09_2_1  normal   9.375980   
2  GSM1823704_252800416877_S01_GE1_107_Sep09_2_3  normal   8.943442   
3  GSM1823705_252800416894_S01_GE1_107_Sep09_1_1  normal   9.020798   
4  GSM1823706_252800416894_S01_GE1_107_Sep09_1_3  normal   8.806154   

   NM_013290  ENST00000322831  NM_001625  lincRNA:chr7:226042-232442_R  \
0   7.718016         6.044438  10.747077                      9.133777   
1   7.072232         6.976741  10.429671                      9.526500   
2   7.964573         6.269055  10.825025                      9.396855   
3   7.824639         6.165165  11.646788                      8.776462   
4   7.555348         6.230969  11.635247                      8.911383   

   NM_032391  ENST00000238571  XR_108906  ...  NM_152343  NM_001005327  \
0   4.735581         5.634732   4.670231  ...   6.3686

In [17]:
X = df.iloc[:, 2:-1].values  # 假设特征在第三列到倒数第二列
y = df['type_encoded'].values
# 标准化特征（可选，但推荐）
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
# 添加截距项
X = np.hstack([np.ones((X.shape[0], 1)), X])
m, n = X.shape
print(X)


[[ 1.         -0.29044738  0.15716683 ...  0.29101254  0.57661691
   1.16698088]
 [ 1.          1.44237276 -0.96712986 ... -1.01273948 -0.75420077
  -0.79655215]
 [ 1.          0.34444946  0.58641694 ... -0.52240203 -0.23294027
   1.1443803 ]
 ...
 [ 1.          0.61844411  0.04204796 ... -0.05398532 -0.51800249
  -0.47654098]
 [ 1.         -1.75940706  0.81641611 ...  1.42053716 -0.62246924
  -0.63175092]
 [ 1.          0.05034099 -0.15467297 ...  1.55371374  0.49169946
   0.03375616]]


In [18]:
def soft_thresholding(x, lambda_):
    return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0)

def ista(X, y, lambda_, alpha, n_iters, tol=1e-4, patience=20):
    m, n = X.shape
    beta = np.zeros(n)
    losses = []
    best_loss = float('inf')
    not_improved_count = 0

    for i in range(n_iters):
        prediction = X.dot(beta)
        error = prediction - y
        current_loss = (1 / (2 * m)) * np.dot(error, error) + lambda_ * np.sum(np.abs(beta))
        losses.append(current_loss)

        # Check for early stopping
        if current_loss < best_loss - tol:
            best_loss = current_loss
            not_improved_count = 0
        else:
            not_improved_count += 1
            if not_improved_count > patience:
                print(f"Early stopping after {i + 1} iterations")
                break

        gradient = X.T.dot(error) / m
        beta = beta - alpha * gradient
        beta = soft_thresholding(beta, lambda_ * alpha)

    return beta, losses

In [19]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import itertools

def cross_validate_ista_accuracy(X, y, lambdas, alphas, k, n_iters):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    best_params = {}
    best_score = 0  # 初始化为0，因为我们希望找到最高的准确率

    # 迭代所有lambda和alpha的组合
    for lambda_, alpha in itertools.product(lambdas, alphas):
        scores = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            beta, _ = ista(X_train, y_train, lambda_, alpha, n_iters)

            # 计算验证集上的准确率
            predictions = X_test.dot(beta)
            predicted_labels = (predictions > 0.5).astype(int)
            accuracy = accuracy_score(y_test, predicted_labels)
            scores.append(accuracy)

        mean_score = np.mean(scores)
        if mean_score > best_score:
            best_score = mean_score
            best_params = {'lambda': lambda_, 'alpha': alpha}

    return best_params, best_score



In [20]:
# 参数范围
lambdas = np.logspace(-4, 1, 6)  # 从1e-4到1e1
alphas = np.logspace(-5, -2, 4)  # 从1e-5到1e-2

# 执行交叉验证
best_params, best_score = cross_validate_ista_accuracy(X, y, lambdas, alphas, k=5, n_iters=1000)

print("Best Parameters:", best_params)
print("Best Validation Score:", best_score)


Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after 22 iterations
Early stopping after

In [21]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#    print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))

# **4 - Data:**

**4.1 Provenance of the datasets**

We chose a dataset for breast cancer classification from the Curated Microarray Database, a repository containing 78 handpicked cancer microarray datasets, extensively curated from 30.000 studies from the Gene Expression Omnibus (GEO), solely for machine learning.

**4.2 Goal of the analysis**

Our goal is that after inputting the characteristic genes, our model can accurately diagnose whether the patient has breast cancer, and the model will output normal or breast cancer.

**4.3 Dataset introduction**

Our dataset contains 289 samples with 35,982 characterized genes and has two diagnostic types, normal and breast_adenocarcinoma. In this dataset, the number of samples in it is much less than the number of features, which is a classic LASSO problem, and also the number of normal and breast_adenocarcinoma is very well balanced as 146:143, so the dataset is effective in avoiding overfitting and false negative problems. Curated Microarray Database has provided the prediction accuracy of many machine learning models for our chosen dataset, 0.93 for SVM, 0.8 for Decision Tree, 0.7 for Multilayer Perceptron, 0.82 for KNN, and 0.78 for K-MEANS, which were able to be compared to our ISTA.


# **5 - References:**

Feltes, B.C.; Chandelier, E.B.; Grisci, B.I.; Dorn, M. CuMiDa: An Extensively Curated Microarray Database for Benchmarking and Testing of Machine Learning Approaches in Cancer Research. Journal of Computational Biology, 2019.

Section 13.4.3 in K. P. Murphy, Machine Learning: A Probabilistic Perspective, MIT, 2012

Section 10.6 in Python Machine Learning: Machine Learning and Deep Learning with Python, scikit-learn, and TensorFlow 2, 3rd Edition 3rd ed. Edition
by Sebastian Raschka, Vahid Mirjalili
